In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import glob
from main_io_comp_vis_all import calibrate_magnetic_wifi_ibeacon_to_position, read_data_file
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
floor_txt_num = {'F1':1,'1F':1,'L1':1,'1L':1,'F2':3,'2F':3,'L2':3,'2L':3,'F3':4,'3F':4,'L3':4,'3L':4,'F4':5,'4F':5,'L4':5,'4L':5,
                'F5':6,'5F':6,'L5':6,'5L':6,'F6':7,'6F':7,'L6':7,'6L':7,'F7':8,'7F':8,'L7':8,'7L':8,'F8':9,'8F':9,'L8':9,'8L':9,
                'F9':10,'9F':10,'L9':10,'9L':10,'F10':11,'10F':11,'L10':11,'10L':11,'F11':12,'11F':12,'L11':12,'11L':12,
                'B1':1,'1B':1,"B":1,'B2':2,'2B':2,'B3':3,'3B':3,'B4':4,'4B':4}
test_paths = glob.glob('../input/indoor-location-navigation/test/*')

In [ ]:
def return_site_id(path):
    with open(path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    for line_data in lines:
        line_data = re.split('[ :\t]',line_data)
        for idx, word in enumerate(line_data):
            if word.lower() == "siteid":
                return line_data[idx+1]     

'''for idx, row in enumerate(mwi_data.wifi):
    test_df.at[idx,row[2]]=int(row[3])

In [ ]:
mall_max_fxy = pd.read_csv('../input/bssid-maxfxy-unique/max_fxy.csv')
sub_file = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')
sub_file[['site','file','time']] = sub_file['site_path_timestamp'].str.split('_', expand=True)
sub_file["time"]=pd.to_numeric(sub_file["time"])
sub_file["site"]=sub_file["site"].astype('str')
site_unique = set(sub_file["site"])
file_unique = set(sub_file["file"])

# Read model, Predict and write Output file

In [ ]:
i=0
for idx_file, file in enumerate(file_unique):
    print(idx_file," out of ",len(file_unique))
    try:
        site = return_site_id('../input/indoor-location-navigation/test/'+file+'.txt')
        model = keras.models.load_model('../input/int-loc-mdl-v1/'+site+'_mdl')
        site_bssid = pd.read_csv('../input/bssid-maxfxy-unique/'+site+'_bssid.csv')
    except:
        i+=1
        continue
    
    mwi_data = read_data_file('../input/indoor-location-navigation/test/'+file+'.txt')    #break
    inp_df = np.ones((len(set(mwi_data.wifi[:,0])),len(site_bssid['0'])+1))*(-100)
    out_df = np.ones((len(set(mwi_data.wifi[:,0])),4))*(-100)
    mwi_data_wifi_df = pd.DataFrame(data=mwi_data.wifi[:,:4],columns=['Time','SSID','BSSID','RSSI'])
    for idx, time in enumerate(set(mwi_data.wifi[:,0])):
        mwi_data_wifi_df_timesnap = mwi_data_wifi_df[mwi_data_wifi_df["Time"]==time][["BSSID","RSSI"]]
        inp_df[idx,0]=time
        for idx_row, row in mwi_data_wifi_df_timesnap.iterrows():
            try:
                inp_df[idx, site_bssid[site_bssid['0']==row["BSSID"]].index.values[0]+1] = row["RSSI"]
            except:
                continue
    inp_df = pd.DataFrame(data=inp_df,columns=(['Time']+list(site_bssid['0'])))
    for col in inp_df.columns[1:]:
        inp_df[col] = (inp_df[col]+100)/100
    out_df[:,1:4] = model.predict(inp_df[inp_df.columns[1:]])
    out_df[:,0]=np.array(inp_df["Time"])
    out_df = pd.DataFrame(data=out_df,columns=(['Time','floor','x','y']))
    out_df = np.ones((len(set(mwi_data.wifi[:,0])),4))*(-100)
    out_df[:,1:4] = model.predict(inp_df[inp_df.columns[1:]])
    out_df[:,0]=np.array(inp_df["Time"])
    out_df = pd.DataFrame(data=out_df,columns=(['Time','floor','x','y']))
    out_df["floor"]=out_df["floor"]*(int(mall_max_fxy[mall_max_fxy["mall"]==site]["max_floor"]))
    out_df["x"]=out_df["x"]*(int(mall_max_fxy[mall_max_fxy["mall"]==site]["max_x"]))
    out_df["y"]=out_df["y"]*(int(mall_max_fxy[mall_max_fxy["mall"]==site]["max_y"]))
    
    min_time = out_df["Time"].min()
    max_time = out_df["Time"].max()
    for idx_sub, row_sub in sub_file[sub_file["file"]==file].iterrows():
        if int(row_sub["time"]) < min_time:
            sub_file.loc[list((sub_file["file"]==file) & (sub_file["time"]==(row_sub["time"]))),["floor","x","y"]] = out_df.loc[list(out_df["Time"]==min_time),["floor","x","y"]].values            
            continue        
        elif int(row_sub["time"]) > max_time:
            sub_file.loc[list((sub_file["file"]==file) & (sub_file["time"]==(row_sub["time"]))),["floor","x","y"]] = out_df.loc[list(out_df["Time"]==max_time),["floor","x","y"]].values   
            continue
        time_ll = int(out_df.loc[list(out_df["Time"]<=row_sub["time"]),["Time"]].max())
        time_hl = int(out_df.loc[list(out_df["Time"]>=row_sub["time"]),["Time"]].min())
        val_ll = out_df.loc[list(out_df["Time"]==time_ll),["floor","x","y"]].values
        val_hl = out_df.loc[list(out_df["Time"]==time_hl),["floor","x","y"]].values
        sub_file.loc[list((sub_file["file"]==file) & (sub_file["time"]==(row_sub["time"]))),["floor","x","y"]] = (val_ll + val_hl)/2

In [ ]:
sub_file[['site_path_timestamp', 'floor', 'x', 'y']].to_csv('Submission_file.csv')

min_time = out_df["Time"].min()
max_time = out_df["Time"].max()
for idx_sub, row_sub in sub_file[sub_file["file"]==file].iterrows():
    if int(row_sub["time"]) < min_time:
        print((sub_file["file"]==file).sum())
        print('-----------------------------------------------')
        print(sub_file.loc[list((sub_file["file"]==file) & (sub_file["time"]==(row_sub["time"]))),["floor","x","y"]])
        print('-----------------------------------------------')
        sub_file.loc[list((sub_file["file"]==file) & (sub_file["time"]==(row_sub["time"]))),["floor","x","y"]] = out_df.loc[list(out_df["Time"]==min_time),["floor","x","y"]].values
        print(sub_file.loc[list((sub_file["file"]==file) & (sub_file["time"]==(row_sub["time"]))),["floor","x","y"]])
        print('-----------------------------------------------')
        continue        
    elif int(row_sub["time"]) > max_time:
        sub_file.loc[list((sub_file["file"]==file) & (sub_file["time"]==(row_sub["time"]))),["floor","x","y"]] = out_df.loc[list(out_df["Time"]==max_time),["floor","x","y"]].values   
        # sub_file[(sub_file["file"]==file) and (sub_file["time"]==int(row_sub["time"]))]["floor","x","y"]=out_df[out_df["Time"]==max_time][["floor","x","y"]]
        continue
    time_ll = int(out_df.loc[list(out_df["Time"]<=row_sub["time"]),["Time"]].max())
    time_hl = int(out_df.loc[list(out_df["Time"]>=row_sub["time"]),["Time"]].min())
    print("HL",time_hl, "\n LL", time_ll)
    val_ll = out_df.loc[list(out_df["Time"]==time_ll),["floor","x","y"]].values
    val_hl = out_df.loc[list(out_df["Time"]==time_hl),["floor","x","y"]].values
    sub_file.loc[list((sub_file["file"]==file) & (sub_file["time"]==(row_sub["time"]))),["floor","x","y"]] = (val_ll + val_hl)/2
    print('---------------------------------------')    
    print((val_ll + val_hl)/2)
    print('---------------------------------------')
    print(sub_file.loc[list((sub_file["file"]==file) & (sub_file["time"]==(row_sub["time"]))),["floor","x","y"]])    
   

mwi_data = read_data_file('../input/indoor-location-navigation/test/2bdf91b83e265e197ac7b117.txt')
site = return_site_id('../input/indoor-location-navigation/test/2bdf91b83e265e197ac7b117.txt')
site_bssid = pd.read_csv('../input/bssid-maxfxy-unique/'+site+'_bssid.csv')
len(site_bssid['0'])
len(set(mwi_data.wifi[:,0]))
inp_df = np.ones((len(set(mwi_data.wifi[:,0])),len(site_bssid['0'])+1))*(-100)
out_df = np.ones((len(set(mwi_data.wifi[:,0])),4))*(-100)
mwi_data_wifi_df = pd.DataFrame(data=mwi_data.wifi[:,:4],columns=['Time','SSID','BSSID','RSSI'])
for idx, time in enumerate(set(mwi_data.wifi[:,0])):
    mwi_data_wifi_df_timesnap = mwi_data_wifi_df[mwi_data_wifi_df["Time"]==time][["BSSID","RSSI"]]
    inp_df[idx,0]=time
    for idx_row, row in mwi_data_wifi_df_timesnap.iterrows():
        try:
            inp_df[idx, site_bssid[site_bssid['0']==row["BSSID"]].index.values[0]+1] = row["RSSI"]
            #i+=1
        except:
            #print(idx, idx_row)
            #j+=1
            continue
#print(i,j)
inp_df = pd.DataFrame(data=inp_df,columns=(['Time']+list(site_bssid['0'])))
for col in inp_df.columns[1:]:
    inp_df[col] = (inp_df[col]+100)/100
model = keras.models.load_model('../input/int-loc-mdl-v1/'+site+'_mdl')
out_df[:,1:4] = model.predict(inp_df[inp_df.columns[1:]])
out_df[:,0]=np.array(inp_df["Time"])
out_df = pd.DataFrame(data=out_df,columns=(['Time','floor','x','y']))
out_df = np.ones((len(set(mwi_data.wifi[:,0])),4))*(-100)
out_df[:,1:4] = model.predict(inp_df[inp_df.columns[1:]])
out_df[:,0]=np.array(inp_df["Time"])
out_df = pd.DataFrame(data=out_df,columns=(['Time','floor','x','y']))
out_df["floor"]=out_df["floor"]*(int(mall_max_fxy[mall_max_fxy["mall"]==site]["max_floor"]))
out_df["x"]=out_df["x"]*(int(mall_max_fxy[mall_max_fxy["mall"]==site]["max_x"]))
out_df["y"]=out_df["y"]*(int(mall_max_fxy[mall_max_fxy["mall"]==site]["max_y"]))